# Build `MESH` Model Setup for the Nelson-Churchill River Basin

We use the `MESHFlow` Python package to build a `MESH` model setup for the Nelson-Churchill River Basin

It's also better to keep the geospatial fabric files next to the "agnostic" step's outputs:

In [3]:
#!cp -r ../1-geofabric/ncrb-geofabric/ /home/fuaday/scratch/ncrb-models/geofabric-outputs/

Let's start by importing the necessary libraries:

In [10]:
# import necessary libraries
import meshflow # version v0.1.0-dev1
import os # python 3.10.2

Now, let's provide necessary information to set up the `MESH` model:

In [11]:
# main work path - modify
work_path = '/home/fuaday/scratch/ncrb-models/'
# using meshflow==v0.1.0-dev1
# modify each segment to match your settings
config = {
    'riv': os.path.join(work_path, 'geofabric-outputs/ncrb-geofabric', 'ncrb_rivers.shp'),
    'cat': os.path.join(work_path, 'geofabric-outputs/ncrb-geofabric', 'ncrb_subbasins.shp'),
    'landcover': os.path.join(work_path, 'gistool-outputs', 'ncrb_model_stats_NA_NALCMS_landcover_2015v2_30m_mod.csv'),
    'forcing_files': os.path.join(work_path, 'easymore-outputs4'),
    'forcing_vars': [ # does the variable list, match those of the "agnostic" step?
        "RDRS_v2.1_P_P0_SFC",
        "RDRS_v2.1_P_HU_09944",
        "RDRS_v2.1_P_TT_09944",
        "RDRS_v2.1_P_UVC_09944",
        "RDRS_v2.1_A_PR0_SFC",
        "RDRS_v2.1_P_FB_SFC",
        "RDRS_v2.1_P_FI_SFC",
    ],
    'forcing_units': { # Here, enter RDRS's original variable units
        'RDRS_v2.1_P_P0_SFC': 'millibar',
        'RDRS_v2.1_P_HU_09944': 'kg/kg',
        'RDRS_v2.1_P_TT_09944': 'celsius',
        'RDRS_v2.1_P_UVC_09944': 'knot',
        'RDRS_v2.1_A_PR0_SFC': 'm/hr',
        'RDRS_v2.1_P_FB_SFC': 'W/m^2',
        'RDRS_v2.1_P_FI_SFC': 'W/m^2',
    },
    'forcing_to_units': { # And here, the units that MESH needs to read
         'RDRS_v2.1_P_UVC_09944': 'm/s',
         'RDRS_v2.1_P_FI_SFC': 'W/m^2',
         'RDRS_v2.1_P_FB_SFC': 'W/m^2',
         'RDRS_v2.1_A_PR0_SFC': 'mm/s',
         'RDRS_v2.1_P_P0_SFC': 'pascal',
         'RDRS_v2.1_P_TT_09944': 'kelvin',
         'RDRS_v2.1_P_HU_09944': 'kg/kg',
    },
    'main_id': 'COMID', # what is the main ID of each river segment? Column name in the `cat` Shapefile
    'ds_main_id': 'NextDownID', # what is the downstream segment ID for each river segment? ditto.
    'landcover_classes': { # these are the classes defined for NALCMS-Landsat 2015 dataset. Is this accurate?      
        0: 'Unknown',
        1: 'Temperate or sub-polar needleleaf forest',
        2: 'Sub-polar taiga needleleaf forest',
        3: 'Tropical or sub-tropical broadleaf evergreen forest',
        4: 'Tropical or sub-tropical broadleaf deciduous forest',
        5: 'Temperate or sub-polar broadleaf deciduous forest',
        6: 'Mixed forest',
        7: 'Tropical or sub-tropical shrubland',
        8: 'Temperate or sub-polar shrubland',
        9: 'Tropical or sub-tropical grassland',
        10: 'Temperate or sub-polar grassland',
        11: 'Sub-polar or polar shrubland-lichen-moss',
        12: 'Sub-polar or polar grassland-lichen-moss',
        13: 'Sub-polar or polar barren-lichen-moss',
        14: 'Wetland',
        15: 'Cropland',
        16: 'Barren lands',
        17: 'Urban',
        18: 'Water',
        19: 'Snow and Ice',
    },
    'ddb_vars': { # the stuff that MESH needs: slope, river length, etc... Let me know if there is any issues here!
        'slope': 'ChnlSlope',
        'lengthkm': 'ChnlLength',
        'Rank': 'Rank',
        'Next': 'Next',
        'landcover': 'GRU',
        'unitarea': 'GridArea',
        'landcover_names': 'LandUse', #
        'order': 'IAK',  #
        'uparea': 'DA',   
        'ncontr': 'IWF'
    },
    'ddb_units': {
        'ChnlSlope': 'm/m',
        'ChnlLength': 'km', # is it in km or m? Please check the units of the Shapefile you created!
        'Rank': 'dimensionless',
        'Next': 'dimensionless',
        'GRU': 'dimensionless',
        'GridArea': 'km^2', # what was the unit of the GridArea, or Shape_Area in the `catchments` Shapefile? #
        'IAK': 'dimensionless', #
        'DA': 'km^2',
        'IWF': 'dimensionless',
    },
    'ddb_to_units': {
        'ChnlSlope': 'm/m',
        'ChnlLength': 'm', # This is what MESH needs, no need to change.
        'Rank': 'dimensionless',
        'Next': 'dimensionless',
        'GRU': 'dimensionless',
        'GridArea': 'm^2', # This is what MESH needs, no need to change. #
        'IAK': 'dimensionless', # 
        'DA': 'km^2',
        'IWF': 'dimensionless',
    },
    'ddb_min_values': {
        'ChnlSlope': 1e-10, # in case there are 0s in the `rivers` Shapefile, we need minimums for certain variables
        'ChnlLength': 1e-3,
        'GridArea': 1e-3,
    },
    'gru_dim': 'NGRU', # change to `NGRU` for 'MESH>=r1860', keep for 'MESH<=1860', for example for r1813.
    'hru_dim': 'subbasin',
    'outlet_value': -9999,
}

We can build an "instance" of the workflow class:

In [12]:
exp1 = meshflow.MESHWorkflow(**config)

And, we can run it using:

In [13]:
exp1.run()

/home/fuaday/.local/lib/python3.10/site-packages/xarray/core/accessor_dt.py:72: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  values_as_series = pd.Series(values.ravel(), copy=False)


Once the run is finished, we can checkout the forcing and drainage database file:

In [14]:
exp1.forcing

<xarray.Dataset>
Dimensions:                (subbasin: 32276, time: 48)
Coordinates:
  * time                   (time) datetime64[ns] 1985-12-30T13:00:00 ... 1986...
  * subbasin               (subbasin) int32 71006628 71006557 ... 71004266
Data variables:
    RDRS_v2.1_P_P0_SFC     (subbasin, time) float64 9.593e+04 ... 1.004e+05
    RDRS_v2.1_P_HU_09944   (subbasin, time) float64 0.0008563 ... 0.0003367
    RDRS_v2.1_P_TT_09944   (subbasin, time) float64 256.2 256.0 ... 248.0 248.1
    RDRS_v2.1_P_UVC_09944  (subbasin, time) float64 5.105 6.136 ... 9.712 9.815
    RDRS_v2.1_A_PR0_SFC    (subbasin, time) float64 1.095e-05 ... 2.755e-06
    RDRS_v2.1_P_FB_SFC     (subbasin, time) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    RDRS_v2.1_P_FI_SFC     (subbasin, time) float64 242.6 240.9 ... 203.7 204.4
    lat                    (subbasin) float64 56.47 56.44 56.5 ... 52.05 56.93
    lon                    (subbasin) float64 -97.87 -97.95 ... -110.4 -92.7
    crs                    int64 1
Attributes:
    author:       University of Calgary
    license:      GNU General Public License v3 (or any later version)
    purpose:      Create forcing .nc file for MESH
    Conventions:  CF-1.6
    history:      Created using MESHFlow version v0.1.0.dev1

In [15]:
exp1.ddb

<xarray.Dataset>
Dimensions:     (subbasin: 32276, NGRU: 16)
Coordinates:
  * subbasin    (subbasin) int64 71006628 71006557 ... 71050687 71004266
Dimensions without coordinates: NGRU
Data variables: (12/13)
    ChnlSlope   (subbasin) float64 1.0 0.0006378 0.0003163 ... 1.0 1.0 0.0001875
    ChnlLength  (subbasin) float64 2.866e+03 1.205e+04 ... 1e+03 7.982e+03
    Rank        (subbasin) int32 1 2 3 4 5 6 ... 32272 32273 32274 32275 32276
    Next        (subbasin) int32 4 4 6 6 10 10 16 16 18 18 ... 0 0 0 0 0 0 0 0 0
    GRU         (subbasin, NGRU) float64 0.2896 0.0 0.06695 ... 0.009607 0.0 0.0
    GridArea    (subbasin) float64 3.95e+07 4.912e+07 ... 1.197e+07 1.173e+08
    ...          ...
    IAK         (subbasin) int64 1 1 1 2 1 2 1 1 1 2 1 ... 2 1 1 1 3 3 1 1 1 1 8
    DA          (subbasin) float64 39.5 49.12 30.98 89.41 ... 1.0 1.0 1.308e+06
    IWF         (subbasin) int64 1 1 1 1 1 1 1 1 1 1 1 ... 2 1 1 1 1 1 1 1 2 2 1
    lat         (subbasin) float64 56.47 56.44 56.5 56.5 ... 52.72 52.05 56.93
    lon         (subbasin) float64 -97.87 -97.95 -97.97 ... -108.5 -110.4 -92.7
    crs         int64 1
Attributes:
    author:       University of Calgary
    license:      GNU General Public License v3 (or any later version)
    purpose:      Create a drainage database .nc file for MESH
    featureType:  point

In [16]:
# create a directory for MESH setup
try:
    os.makedirs('/home/fuaday/scratch/ncrb-models/MESH-ncrb')
except FileExistsError:
    pass
# saving drainage database and forcing files
exp1.save('/home/fuaday/scratch/ncrb-models/MESH-ncrb/')

Now that the two important files are built, we can copy a series of setting files into the output path:

In [ ]:
!cp -r setting_files/* /home/fuaday/scratch/ncrb-models/MESH-ncrb/
!mkdir -p /home/fuaday/scratch/ncrb-models/MESH-ncrb/results/

In [3]:
import netCDF4 as nc
import shutil

# Path to the original NetCDF file
ddbnetcdf_path = '/home/fuaday/scratch/ncrb-models/MESH-ncrb/MESH_drainage_database_org.nc'
# Path where the modified NetCDF file will be saved
ddbnetcdf_pathsave = '/home/fuaday/scratch/ncrb-models/MESH-ncrb/MESH_drainage_database.nc'
# First, copy the original NetCDF file to the new location to ensure we don't overwrite the original accidentally
shutil.copyfile(ddbnetcdf_path, ddbnetcdf_pathsave)
# Now open the copied NetCDF file in read-write mode
dataset = nc.Dataset(ddbnetcdf_pathsave, 'r+')
# Access the 'ChnlSlope' variable
chnlslope = dataset.variables['ChnlSlope']

# Find where 'ChnlSlope' is greater than 0.003 and set those values to 0.0001
chnlslope_values = chnlslope[:]
chnlslope_values[chnlslope_values > 0.03] = 0.0001
chnlslope[:] = chnlslope_values  # Update the array in the dataset

# Close the dataset to write changes to the file
dataset.close()
print("Updated NetCDF file saved at:", ddbnetcdf_pathsave)

Updated NetCDF file saved at: /home/fuaday/scratch/ncrb-models/MESH-ncrb/MESH_drainage_database.nc


____